In [161]:
import numpy as np
import sib_ldsc_z as ld
from scipy.optimize import minimize
from scipy.special import comb
from scipy.misc import derivative
import scipy.stats
from importlib import reload
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
import glob
import numba
from numba import jit, njit, prange
reload(ld)

<module 'sib_ldsc_z' from 'c:\\Users\\Hariharan\\Documents\\git_repos\\SNIPar\\ldsc_reg\\inferz\\sib_ldsc_z.py'>

In [162]:
np.random.seed(123)

N = int(1e4)
S = np.array([[[1e-4, -5 * 1e-5], [-5 * 1e-5, 1e-4]]] * N)
V = np.array([[0.5, 0.25], [0.25, 0.5]])

In [163]:
# base functions
def extract_upper_triangle(x):
    
    # =============================== #
    # Extracts the upper triangular portion of 
    # a symmetric matrix
    # =============================== #
    
    n, m = x.shape
    assert n == m
    
    upper_triangle = x[np.triu_indices(n)]
    
    return upper_triangle

def return_to_symmetric(triangle_vec, final_size):
    
    # =============================== #
    # Given a vector of the upper triangular matrix,
    # get back the symmetric matrix
    # =============================== #
    
    X = np.zeros((final_size,final_size))
    X[np.triu_indices(X.shape[0], k = 0)] = triangle_vec
    X = X + X.T - np.diag(np.diag(X))
    
    return X

def extract_bounds(n):
    
    # =============================== #
    # From a number n, the function
    # outputs a list of bounds
    # for a var cov matrix of size
    # n x n
    # =============================== #
    
    # extract idx of flat array whcih are diagonals
    uptriangl_idx = np.array(np.triu_indices(n))
    diags = uptriangl_idx[0, :] == uptriangl_idx[1, :]
    
    # Construct list of bounds
    bounds_list = np.array([(None, None)] * len(diags))
    bounds_list[diags] = (1e-6, None)
    
    bounds_list_out = [tuple(i) for i in bounds_list]
    
    return bounds_list_out


def delete_obs_jk(var, start_idx, end_idx, end_cond):

    # ============================== #
    # var: numpy array
    # end_cond : boolean
    # Function helps take out observations
    # for a jackknife routine
    # ============================== #

    if end_cond:

        var_jk = np.delete(var, range(start_idx, end_idx), 
                             axis = 0)

    else:

        var_jk = np.delete(var, range(start_idx, var.shape[0]), 
                             axis = 0)
        var_jk = np.delete(var_jk, range(end_idx - var.shape[0]))
        
    return var_jk


@njit
def normalize_S(S, norm):
    '''
    A function which normalizes a vector of S matrices
    '''
    
    N = S.shape[0]
    S_norm = np.zeros_like(S)
    for idx in range(N):
        
        Si = S[idx]
        normi = norm[idx]
        S_norm[idx] = Si * normi
    
    return S_norm

@njit
def calc_inv_root(S):
    '''
    A stable solver for S^{-1/2}
    '''
    
    if ~np.any(np.isnan(S)):
        S_eig = np.linalg.eig(S)
        l = np.zeros(S.shape)
        np.fill_diagonal(l,np.power(S_eig[0],-0.5))
        S_inv_root = S_eig[1].dot(np.dot(l,S_eig[1].T))
    else:
        S_inv_root =  np.empty_like(S)
        S_inv_root[:] = np.nan
    return S_inv_root

    
@njit
def standardize_mat(V, S, M):
    '''
    Standardizes V and S matrices by constructing
    D = M [[1/sigma_1, 0], [0, 1/sigma_2]]. sigma_1 and sigma_2
    come from the S matrix provided.
    
    Then Vnew = Dmat @ V @ Dmat
    and Snew = Dmat @ S @ Dmat
    '''
    
    sigma1 = np.sqrt(M * S[0, 0])
    sigma2 = np.sqrt(M * S[1, 1])
    
    Dmat = np.sqrt(M) * np.array([[1/sigma1, 0], 
                                [0, 1/sigma2]])
    Snew = Dmat @ S @ Dmat
    Vnew = Dmat @ V @ Dmat
    
    return Vnew, Snew

@njit
def V2Vmat(V, M):
    '''
    Transforms a 1 dimensional V array
    with 3 elements v1, v2 and r
    into a V matrix
    '''
    # getting important scalars
    v1 = V[0]
    v2 = V[1]
    r = V[2]

    Vmat = (1/M) * np.array([[v1, r * np.sqrt(v1 * v2)], [r * np.sqrt(v1 * v2), v2]])

    return Vmat

@njit 
def Vmat2V(Vmat, M):
    '''
    Makes a 2x2 V matrix
    into a 1 dimensional V
    array containing v1, v2 and
    r
    '''

    v1 = M * Vmat[0, 0]
    v2 = M * Vmat[1, 1]
    r = M * Vmat[0, 1]/np.sqrt(v1 * v2)
    r_check = M * Vmat[1, 0]/np.sqrt(v1 * v2)

    assert r == r_check

    return np.array([v1, v2, r])


In [164]:
def simdata(V, S, N, simr = False):
    """
    Simulates data for z scores.
    
    Inputs:
    V = varcov matrix of true effects
    N = Number of obs/SNPs to generate
    simr = boolean indicating if we want
            to simulate ldscores
            
    
    Outputs:
    None
    
    - It creates an object within the class
    called z
    """
    

    zhat_vec = np.empty((N, V.shape[1]))
    for i in range(N):
        
        Si = S[i]
        
        V = np.array(V)
        Si = np.array(Si)

        # get shape of V
        d = V.shape[0]
        zeromat = np.zeros(d)
        Vnew, Snew = standardize_mat(V, Si, N)

        # generate true effect vector
        sim = np.random.multivariate_normal(zeromat, Snew + Vnew)
        
        # Append to vector of effects
        zhat_vec[i, :] = sim

    return zhat_vec

z = simdata(V/N, S, N)
r = np.ones(N)

In [165]:
def get_logll_scipy(V, z, S, r, N):

    '''
    Gets log likelihood function from scipy
    Used to test if our log likelihood function 
    is defined correctly
    '''

    Vmat = V2Vmat(V, N)
    
    Vnew, Snew = standardize_mat(Vmat, S, N)

    dist = scipy.stats.multivariate_normal(mean = None,
                                          cov = Vnew + Snew)

    nlogll = dist.logpdf(z)
    return nlogll
    

In [166]:
get_logll_scipy(Vmat2V(V/N, N), z[0, :], S[0], r[0], N)

-3.3159026093560375

In [167]:
@njit
def _log_ll(V, z, S, r, N):

    """
    Returns the log likelihood matrix for a given SNP i as formulated by:

    .. math::
        l_i = -\frac{d}{2} log (2 \pi) - \frac{1}{2} log ( |I + r_i S_i^{-1/2} V S_i^{-1/2}| ) -
                \frac{1}{2} z_i^T (I + r_i S_i^{-1/2} V S_i^{-1/2}) ^{-1} z_i

    Inputs:
    V = dxd numpy matrix
    z = dx1 numpy matrix
    S = dxd numpy matrix
    r = scalar
    f = scalar

    Outputs:
    logll = scalar
    """
    Vmat = V2Vmat(V, N)

    Vnew, Snew = standardize_mat(Vmat, S, N)
    Sigma = Snew + Vnew
    logdet = np.linalg.slogdet(Sigma)

    det = np.linalg.det(Sigma)
    if det > 1e-6 or det < -1e-6:
        Sigma_inv = np.linalg.inv(Sigma)
    else:
        Sigma_inv = np.linalg.pinv(Sigma)

    d = Vmat.shape[0]
    z = z.reshape(d,1)

    L = - (d/2) * np.log(2 * np.pi) \
        - (1/2) * logdet[0]*logdet[1] \
        - (1/2) * z.T @ Sigma_inv @ z

    return L[0, 0]

In [168]:
_log_ll(Vmat2V(V/N, N), z[0, :], S[0], r[0], N)

-3.315902609356038

In [169]:
boolcond = np.allclose(get_logll_scipy(Vmat2V(V/N, N), z[0, :], S[0], r[0], N), _log_ll(Vmat2V(V/N, N), z[0, :], S[0], r[0], N))

print("Log likelihood seems to be correctly defined") if boolcond else print("Log likelihood isn't correctly defined")

Log likelihood seems to be correctly defined


In [170]:
@njit
def _grad_ll_v(V, z, S, r, N):

    """
    """

    Vmat = V2Vmat(V, N)
    d = S.shape[0]

    Vnew, Snew = standardize_mat(Vmat, S, N)
    Sigma = Snew + Vnew
    
    # getting important scalars
    v1 = V[0]
    v2 = V[1]
    r = V[2]

    rs = Snew[0, 1]

    sigma1 = np.sqrt(N * S[0, 0])
    sigma2 = np.sqrt(N * S[1, 1])

    sigma1sq = sigma1 ** 2
    sigma2sq = sigma2 ** 2
    
    assert len(z.shape) == 1
    
    z1 = z[0]
    z2 = z[1]
    
    z1sq = z1 ** 2
    z2sq = z2 ** 2

    det = np.linalg.det(Sigma)

    T = z1sq * (1 + (v2/sigma2sq)) - \
            2 * z1 * z2 * (rs + r * np.sqrt(v1 * v2)/(sigma1 * sigma2)) + \
            z2sq * (1 + v1/sigma1sq)

    # gradient wrt v1
    ddet_dv1 = 1/sigma1sq * (1 + v2/sigma2sq) - (r/(sigma1 * sigma2)) * \
            (rs * np.sqrt(v2/v1) + r * v2/(sigma1 * sigma2)) 

    dT_dv1 = (z2sq/sigma1sq) - (r * z1 * z2)/(sigma1 * sigma2) * np.sqrt(v2/v1)

    dl_dv1 = -(1/2) * 1/det * (ddet_dv1 * (1 - T/det) + dT_dv1)

    # gradient wrt v2
    ddet_dv2 = 1/sigma2sq * (1 + v1/sigma1sq) - (r/(sigma1 * sigma2)) * \
            (rs * np.sqrt(v1/v2) + r * v1/(sigma1 * sigma2)) 

    dT_dv2 = (z1sq/sigma2sq) - (r * z1 * z2)/(sigma1 * sigma2) * np.sqrt(v1/v2)

    dl_dv2 = -(1/2) * 1/det * (ddet_dv2 * (1 - T/det) + dT_dv2)

    # gradient wrt r
    ddet_dr = -2 * np.sqrt(v1 * v2)/(sigma1 * sigma2) * \
         (rs + (r * np.sqrt(v1 * v2))/(sigma1 * sigma2))

    dT_dr = -2 * np.sqrt(v1 * v2)/(sigma1 * sigma2) * z1 * z2

    dl_dr = -(1/2) * 1/det * (ddet_dr * (1 - T/det) + dT_dr)

    return np.array([dl_dv1, dl_dv2, dl_dr])

In [171]:
_grad_ll_v(Vmat2V(V/N, N), z[0, :], S[0], r[0], N)

array([ 0.37720196, -0.3548698 , -0.02657381])

In [172]:
@njit
def _num_grad_V(V, z, S, r, N):
    """
    Returns numerical gradient vector of self._log_ll
    Mostly meant to check if self._grad_ll_v is working
    properly
        
    Inputs:
    V = dxd numpy matrix
    z = dx1 numpy matrix
    S = dxd numpy matrix
    u = 1 numpy matrix
    r = 1 numpy matrix
    f = 1 numpy matrix
        
    Outputs:
    g = dxd matrix 
    """
    
    g = np.zeros(V.shape)

    for i in range(0,V.shape[0]):
        dV = np.zeros(V.shape)
        dV[i] = 10 ** (-6)
        V_upper = V+dV
        V_lower = V-dV
        g[i] = (_log_ll(V_upper, z, S, r, N) - \
                    _log_ll(V_lower, z, S, r, N)) / (2 * 10 ** (-6))
    return g

In [173]:
_num_grad_V(Vmat2V(V/N, N), z[0, :], S[0], r[0], N)

array([ 0.37720196, -0.3548698 , -0.02657381])

In [174]:
np.allclose(_num_grad_V(Vmat2V(V/N, N), z[0, :], S[0], r[0], N),
            _grad_ll_v(Vmat2V(V/N, N), z[0, :], S[0], r[0], N))


True

In [175]:
@njit(parallel = True)
def neg_logll_grad(V, 
                   z, S, 
                   u, r):

    """
    Returns the loglikelihood and its gradient wrt V for a given SNP i as formulated by:

    .. math::
        l_i = -\frac{d}{2} log (2 \pi) - \frac{1}{2} log ( |I + r_i S_i^{-1/2} V S_i^{-1/2}| ) -
                \frac{1}{2} z_i^T (I + r_i S_i^{-1/2} V S_i^{-1/2}) ^{-1} z_i

    and

    .. math::
        \frac{dl}{dV} = S^{-1/2} \Sigma_i^{-1} (\Sigma - z_i z_i^T) \Sigma_i^{-1} S^{-1/2}

    Inputs:
    V = dxd numpy matrix
    z = dxN numpy matrix
    S = dxd numpy matrix
    u = 1 numpy matrix
    r = 1 numpy matrix
    f = 1 numpy matrix
    logllfunc = function which calculates logll
                (uses self._log_ll by default)
    gradfunc = function which calculated grad of logll
                (uses self._grad_ll_v by default)

    Outputs:
    -log_ll = 1x1 scalar
    -Gvec = dxd numpy matrix
    """

    # Unflatten V into a matrix
    d = S[0].shape[0]
    N = len(S)
    
    Gvec = np.zeros((N, 3))
    log_ll = np.zeros(N)

    for i in prange(N):

        Si = S[i]
        zi = z[i, :]
        ui = u[i]
        ri = r[i]

        # Si = N * Si

        log_ll[i] = (1/ui) * _log_ll(V, zi, Si, ri, N)
        Gvec[i, :] = (1/ui) * _grad_ll_v(V, zi, Si, ri, N)  #_num_grad_V

    return -log_ll.sum() , -Gvec.sum(axis = 0)

def neglike_wrapper(V, z, S, u, r, f):
    
    '''
    Wrapper for neg_logll_grad to convert V from an
    array of individual parameters to a symmetric
    matrix and solve for the negative log likelihood
    '''

    d = S[0].shape[0]
    N = S.shape[0]
    
    normalizer = 2 * f  * (1 - f) if f is not None else np.ones(N)
    S = normalize_S(S, normalizer)
    
    logll, Gvec = neg_logll_grad(V, 
                               z, S, 
                               u, r)
    
    print(f"Logll : {logll}")
    print(f"V : {V}")
    
    return logll, Gvec

In [176]:
neg_logll_grad(Vmat2V(V/N, N), z, S, r, r)

(32300.330467425734, array([104.25864396, -76.77297078,  24.49580842]))

In [177]:
neglike_wrapper(Vmat2V(V/N, N), z, S, r, r, None)

Logll : 32300.330467425734
V : [0.5 0.5 0.5]


(32300.330467425734, array([104.25864396, -76.77297078,  24.49580842]))

In [178]:
# solving
result = minimize(
            neglike_wrapper, 
            np.ones(3),
            jac = True,
            args = (z, S, r, r, None),
            bounds = [(1e-6, None), (1e-6, None), (-1, 1)],
            method = 'L-BFGS-B',
            options = {'ftol' : 1e-20}
        )

print(results)

Logll : 33351.59910626821
V : [1. 1. 1.]
Logll : 32335.948538664936
V : [0.59175198 0.59175198 0.18350315]
Logll : 32311.79959838797
V : [0.54369924 0.6137565  0.29436543]
Logll : 32298.683663287156
V : [0.44622699 0.56091267 0.42845312]
Logll : 32296.727459379115
V : [0.47353369 0.53963911 0.48372808]
Logll : 32296.483385116346
V : [0.46400225 0.54243472 0.47605599]
Logll : 32296.4829140356
V : [0.46342091 0.54254395 0.47634488]
Logll : 32296.482905947763
V : [0.46338287 0.54258003 0.47642612]
Logll : 32296.482905854777
V : [0.46338225 0.54258368 0.47643514]
Logll : 32296.482905854755
V : [0.4633823  0.54258363 0.47643523]


In [182]:
estimated_parameters = dict(
    v1 = result.x[0],
    v2 = result.x[1],
    r = result.x[2]
)

print(estimated_parameters)

{'v1': 0.46338229858454855, 'v2': 0.5425836332925408, 'r': 0.4764352254823297}
